# Implementation of Algorithm Distillation

## Установка нужных библиотек

In [1]:
!pip install wandb
!pip install stable_baselines3
!pip install shimmy
!pip install gymnasium stable-baselines3
!pip install gym

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cu

## Вход в учетную запись Weights&Biases

In [2]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##Импорт нужных модулей


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from transformers import GPT2Config, GPT2Model
import json
import pickle

import gym
import numpy as np
import wandb

/usr/local/lib/python3.10/dist-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)


### Объявляем модель -- трансформер

In [4]:
class PolicyTransformerConfig(GPT2Config):
    """
    Конфигурация PolicyTransformer.
    """
    def __init__(
            self,
            hidden_size=128,
            state_dim=0,
            act_dim=0,
            act_num=0,
            max_ep_len=20,
            context_len=80,
            token_mask_prob=0.3,
            **kwargs
    ):
        """
        Args:
            state_dim: размерность пространства состояний
            act_dim: размерность пространства действий
            max_ep_len: максимальная длина эпизода

            act_num: количество действий в домене
            context_len: длина контекста, используемого в качестве входных данных для модели
            token_mask_prob: вероятность маскирования токена во время обучения


        """
        super().__init__(**kwargs)

        self.hidden_size = hidden_size

        self.state_dim = state_dim
        self.act_dim = act_dim
        self.act_num = act_num
        self.max_ep_len = max_ep_len

        self.context_len = context_len
        self.token_mask_prob = token_mask_prob


class PolicyTransformer(nn.Module):
    """
    Transformer, который предсказывает следующее действие на основе истории, с GPT2 в кач-ве backbone.

    s_0, a_0, r_0,   s_1, a_1, r_1,  ...  s_T, a_T (masked), r_T (masked)

                                           |
                                           V

                                  [logit_1  logit_2  ...  logit_|A|]

                                           |
                                           V

                            Output (действие с максимальной вероятностью)
    """
    def __init__(self, config: GPT2Config, *args):
        super().__init__(*args)

        self.config = config

        self.hidden_size = config.hidden_size
        self.encoder = GPT2Model(config)

        self.embed_state = torch.nn.Linear(config.state_dim, config.hidden_size)
        self.embed_action = torch.nn.Linear(config.act_dim, config.hidden_size)
        self.embed_reward = torch.nn.Linear(1, config.hidden_size)
        self.embed_timestep = nn.Embedding(config.max_ep_len, config.hidden_size)

        self.predict_action = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.ReLU(),
            nn.Linear(config.hidden_size, config.act_num),
        )

    def forward(
            self,
            states,
            actions,
            rewards,
            timesteps,
            attention_mask = None,
            **kwargs,
    ) -> dict:
        batch_size, seq_length = states.shape[0], states.shape[1]

        if attention_mask is None:
            attention_mask = torch.ones((batch_size, seq_length), dtype=torch.long)

        state_embeddings = self.embed_state(states) # (batch_size, seq_length, hidden_size)
        action_embeddings = self.embed_action(actions) # (batch_size, seq_length, hidden_size)
        reward_embeddings = self.embed_reward(rewards) # (batch_size, seq_length, hidden_size)
        time_embeddings = self.embed_timestep(timesteps) # (batch_size, seq_length, hidden_size)

        # эмбеддинги времени обрабатываются как positional
        state_embeddings = state_embeddings + time_embeddings
        action_embeddings = action_embeddings + time_embeddings
        reward_embeddings = reward_embeddings + time_embeddings

        # последовательность теперь выглядит как (s_1, a_1, r_1, s_2, a_2, ...)
        # что хорошо работает в авторегрессии, т.к. состояния предсказывают действия
        stacked_inputs = (
            torch.stack((state_embeddings, action_embeddings, reward_embeddings), dim=1) # (batch_size, 3, seq_length, hidden_size)
            .permute(0, 2, 1, 3) # (batch_size, seq_length, 3, hidden_size)
            .reshape(batch_size, 3 * seq_length, self.hidden_size)
        )

        device = stacked_inputs.device

        stacked_attention_mask = (
            torch.stack((attention_mask, attention_mask, attention_mask), dim=1) # (batch_size, 3, seq_length)
            .permute(0, 2, 1) # (batch_size, seq_length, 3)
            .reshape(batch_size, 3 * seq_length) # (batch_size, 3 * seq_length)
        )

        if self.training and self.config.token_mask_prob > 0:
            mask = (torch.rand(stacked_attention_mask.shape) > self.config.token_mask_prob).float().to(device)
            stacked_attention_mask = stacked_attention_mask * mask

        # всегда маскируем эмбеддинг вознаграждения во время последнего шага
        stacked_attention_mask[:, -2:] = 0

        # подаем в модель входные эмбеддинги (не индексы слов, как в NLP)
        encoder_outputs = self.encoder(
            inputs_embeds=stacked_inputs,
            attention_mask=stacked_attention_mask,
        )
        x = encoder_outputs[0]

        x = x.reshape(batch_size, seq_length, 3, self.hidden_size).permute(0, 2, 1, 3) # (batch_size, 3, seq_length, hidden_size)

        # используем эмбеддинг состояния на последнем шаге в кач-ве input, получаем распределение действий
        action_pred = self.predict_action(x[:, 0, -1])  # (batch_size, act_num)

        action_true = actions[:, -1, 0].long()  # (batch_size,)

        # получаем логарифмические вероятности прeдсказанных действий
        log_probs = F.log_softmax(action_pred, dim=1)
        nll_loss = F.nll_loss(log_probs, action_true)

        return {'action_pred': action_pred, 'loss': nll_loss}

    def predict(
            self,
            states=None,
            actions=None,
            rewards=None,
            timesteps=None,
            attention_mask=None,
            temperature=0.0,
            **kwargs,
    ):
        model_output = self.forward(states, actions, rewards, timesteps, attention_mask, **kwargs)

        if temperature == 0.:
            # если температура не инициализирована, то по argmax
            action_idx = torch.argmax(model_output['action_pred'], dim=1)
        else:
            # иначе, по распределению
            action_idx = torch.multinomial(F.softmax(model_output['action_pred'] / temperature, dim=1), num_samples=1)

        return action_idx.item()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


###Функция для нахождения конфига DarkRoom

In [5]:
def find_config_file(env_id: str, alg: str):
    if env_id.startswith('DarkRoom'):
        return f"configs/DarkRoom-{alg}.json"
    else:
        raise ValueError(f"Неизвестная среда или алгоритм: {env_id, alg}")

In [6]:
!git clone https://github.com/me1nna/in-context-ad.git
# клоню со своего репозитория, вытаскиваю из папки data архив data.zip

Cloning into 'in-context-ad'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 119 (delta 9), reused 20 (delta 9), pack-reused 99
Receiving objects: 100% (119/119), 441.01 MiB | 21.33 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Updating files: 100% (30/30), done.


## В архиве -- датасет, envs, configs



In [7]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/get_data.ipynb     
  inflating: data/darkroom_normal_117.pkl  
  inflating: data/darkroom_normal_105.pkl  
  inflating: data/darkroom_normal_67.pkl  
  inflating: data/darkroom_normal_16.pkl  
  inflating: data/darkroom_normal_160.pkl  
  inflating: data/darkroom_normal_113.pkl  
  inflating: data/darkroom_normal_163.pkl  
  inflating: data/darkroom_normal_196.pkl  
  inflating: data/darkroom_normal_10.pkl  
  inflating: data/darkroom_normal_170.pkl  
  inflating: data/darkroom_normal_60.pkl  
  inflating: data/darkroom_normal_6.pkl  
  inflating: data/darkroom_normal_46.pkl  
  inflating: data/darkroom_normal_77.pkl  
  inflating: data/darkroom_normal_188.pkl  
  inflating: data/darkroom_normal_38.pkl  
  inflating: data/darkroom_normal_148.pkl  
  inflating: data/darkroom_normal_162.pkl  
  inflating: data/darkroom_normal_20.pkl  
  inflating: data/darkroom_normal_199.pkl  
  inflating: data/darkroom_normal_171.pkl  
  inflatin

## Подготовка датасета

In [8]:
from torch.utils.data import Dataset


class LifetimeDataset(Dataset):
    def __init__(self, lifetimes: list, context_len: int = 3):
        """
        args:
            lifetimes: Список, где каждый жизненный цикл представляет собой список траекторий
            context_len: Длина предыдущего контекста для предсказания
        """
        self.lifetimes = lifetimes
        self.context_len = context_len

        self.lifetime_lens = [len(trajs) - 1 for trajs in self.lifetimes]
        self.total_len = sum(self.lifetime_lens)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        """
        Получает контекст до индекса idx
        """

        # находим индекс жизненного цикла и индекс шага
        for lifetime_idx in range(len(self.lifetimes)):
            if idx < self.lifetime_lens[lifetime_idx]:
                break
            idx -= self.lifetime_lens[lifetime_idx]

        step_idx = idx + 1

        # получаем context_len предыдущих шагов как контекст в том же жизненном цикле
        context = self.lifetimes[lifetime_idx][max(0, step_idx - self.context_len):step_idx]

        return {
            'states': [s[0] for s in context],
            'actions': [s[1] for s in context],
            'rewards': [s[2] for s in context],
            'timesteps': [s[3] for s in context],
        }

In [9]:
import os
import pickle
from tqdm.auto import tqdm

def collect_from_pkl(file_prefix: str, data_dir: str = "data", subsample_gap: int = 1, max_lifetimes: int = 1500):
    """
    Функция для получения всех жизненных циклов
    """
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Directory not found: {data_dir}")

    filenames = []
    lifetimes = []

    for file in os.listdir(data_dir):
        if file.startswith(file_prefix) and file.endswith('.pkl'):
            filenames.append(os.path.join(data_dir, file))

    print(f'Найдено {len(filenames)} файлов в директории {data_dir}:')

    for filename in tqdm(filenames):
        if len(lifetimes) >= max_lifetimes:
            break
        try:
            with open(filename, 'rb') as f:
                lifetime = pickle.load(f)
                lifetime = lifetime[::subsample_gap]
                lifetimes.append(lifetime)
        except EOFError:
            print(f'Не получилось загрузить {filename}.')

    print(f'Получено {len(lifetimes)} жизненных циклов.')

    return lifetimes

In [10]:
def collate_fn(batch):

    states = [torch.FloatTensor(s['states']) for s in batch]
    actions = [torch.FloatTensor(s['actions']) for s in batch]
    rewards = [torch.FloatTensor(s['rewards']) for s in batch]
    timesteps = [torch.tensor(s['timesteps']) for s in batch]

    max_length = max(len(s) for s in states)

    attention_mask = [torch.tensor([0] * (max_length - len(s['states']))
                                   + [1] * len(s['states']))
                      for s in batch]

    for idx in range(len(batch)):
        if len(states[idx]) < max_length:
            # если данные короче максимальной длины в батче, добавляем паддинг слева
            pad = (max_length - len(states[idx]), 0)

            states[idx] = F.pad(states[idx], (states[idx].dim() - 1) * (0, 0) + pad)
            actions[idx] = F.pad(actions[idx], (actions[idx].dim() - 1) * (0, 0) + pad)
            rewards[idx] = F.pad(rewards[idx], pad)
            timesteps[idx] = F.pad(timesteps[idx], pad)

        if actions[idx].dim() == 1: # (seq_len,)
            # действия тут -- скалярны, поэтому создаем dummy измерение
            actions[idx] = actions[idx].unsqueeze(-1)

        # dummy измерение для наград
        rewards[idx] = rewards[idx].unsqueeze(-1)

    return {
        'states': torch.stack(states), # (batch_size, seq_len, state_dim)
        'actions': torch.stack(actions), # (batch_size, seq_len, action_dim)
        'rewards': torch.stack(rewards), # (batch_size, seq_len, 1)
        'timesteps': torch.stack(timesteps), # (batch_size, seq_len)
        'attention_mask': torch.stack(attention_mask), # (batch_size, seq_len)

        'return_loss': True,
    }


In [11]:
def prepare_for_prediction(
        states: list,
        actions: list,
        rewards: list,
        timesteps: list,
        env: gym.Env,
        context_len: int = 10,
        device: str = 'cuda',
):
    """
    Подготавливаем данные для предсказаний
    """
    datum = {
        'states': states[-context_len:],
        'actions': actions[-context_len + 1:] + [env.action_space.sample()],
        'rewards': rewards[-context_len + 1:] + [0],
        'timesteps': timesteps[-context_len:],
    }
    model_input = collate_fn([datum])
    model_input['return_loss'] = False

    for k in model_input:
        if hasattr(model_input[k], 'to'):
            model_input[k] = model_input[k].to(device)

    return model_input

# Eval

In [13]:
@torch.no_grad()
def eval_model(
        env_id: str,
        lifetime_num: int,
        episode_num: int,
        model_config: str,
        model_path: str,
        device: str,
        log_interval: int = 100,
        temperature: float = 1.,
        baseline: str = None,
) -> None:
    # Указываем путь к конфигурационному файлу в папке data/configs
    config_path = os.path.join('data', model_config)
    config = PolicyTransformerConfig.from_json_file(config_path)
    model = PolicyTransformer(config)

    # Проверка существования файла модели
    model_file_path = os.path.join(model_path, 'model.safetensors')
    if not os.path.exists(model_file_path):
        raise FileNotFoundError(f"Model file not found: {model_file_path}")

    # Загрузка модели
    from safetensors.torch import load_file
    model_state_dict = load_file(model_file_path)
    model.load_state_dict(model_state_dict)
    model.eval()
    model.to(device)

    context_len = model.config.context_len

    states = []
    actions = []
    rewards = []
    timesteps = []

    wandb.init(
        project='alg-distill',
        name=env_id + '-eval',
        config=config.to_dict(),
    )

    for lifetime_idx in range(lifetime_num):
        env = gym.make(env_id)
        ep_len = env.episode_length

        for i in range(episode_num):
            traj = []
            obs = env.reset()
            for t in range(ep_len):
                states += [obs]
                timesteps += [t]
                if baseline == 'random':
                    action = env.action_space.sample()
                else:
                    model_input = prepare_for_prediction(states, actions, rewards, timesteps,
                                                        env, context_len, device)
                    action = model.predict(**model_input, temperature=temperature) # temperature > 0 to allow exploration
                new_obs, reward, done, _ = env.step(action)
                actions += [action]
                rewards += [reward]
                traj += [(obs, action, reward, t)]
                obs = new_obs

                if done:
                    break

            mean_reward = np.mean(rewards[-ep_len:])
            print(f"Lifetime {lifetime_idx}, episode {i}, mean reward {mean_reward}")
            if (i+1) % log_interval == 0:
                log_interval_mean_reward = np.mean(rewards[-log_interval:])
                wandb.log({f'mean_reward': log_interval_mean_reward})
                env.render(trajectory=traj,
                           log_name=f'l{lifetime_idx}/e{i:05}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
from tqdm import tqdm


model_config = find_config_file(env_id='DarkRoom-v0', alg='dt')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'Using device: {device}')
print(f'Using model config: {model_config}')

Using device: cuda
Using model config: configs/DarkRoom-dt.json


### Регистрируем среду 😝




In [15]:
import sys

sys.path.append(os.path.abspath('data/envs'))

from darkroom import DarkRoom
from gym.envs.registration import register

register(
    id='DarkRoom-v0',
    entry_point='darkroom:DarkRoom',
    max_episode_steps=100,
)

## Распакуем последние чекпоинты!

In [16]:
!unzip in-context-ad/outputs/output_final.zip

Archive:  in-context-ad/outputs/output_final.zip
   creating: checkpoint-69575/
  inflating: checkpoint-69575/scheduler.pt  
  inflating: checkpoint-69575/model.safetensors  
  inflating: checkpoint-69575/optimizer.pt  
  inflating: checkpoint-69575/training_args.bin  
  inflating: checkpoint-69575/rng_state.pth  
  inflating: checkpoint-69575/trainer_state.json  


In [18]:
eval_model(
            env_id='DarkRoom-v0',
            lifetime_num=20,
            episode_num=2000,
            model_config=model_config,
            model_path='checkpoint-69575',
            temperature=1.,
            device=device,
        )

wandb: Currently logged in as: innavakulenko (innavakulenko-none). Use `wandb login --relogin` to force relogin


Выходные данные были обрезаны до нескольких последних строк (5000).
Lifetime 17, episode 1002, mean reward 0.2
Lifetime 17, episode 1003, mean reward 0.05
Lifetime 17, episode 1004, mean reward 0.15
Lifetime 17, episode 1005, mean reward 0.0
Lifetime 17, episode 1006, mean reward 0.0
Lifetime 17, episode 1007, mean reward 0.1
Lifetime 17, episode 1008, mean reward 0.0
Lifetime 17, episode 1009, mean reward 0.05
Lifetime 17, episode 1010, mean reward 0.0
Lifetime 17, episode 1011, mean reward 0.15
Lifetime 17, episode 1012, mean reward 0.0
Lifetime 17, episode 1013, mean reward 0.15
Lifetime 17, episode 1014, mean reward 0.0
Lifetime 17, episode 1015, mean reward 0.0
Lifetime 17, episode 1016, mean reward 0.1
Lifetime 17, episode 1017, mean reward 0.0
Lifetime 17, episode 1018, mean reward 0.05
Lifetime 17, episode 1019, mean reward 0.0
Lifetime 17, episode 1020, mean reward 0.15
Lifetime 17, episode 1021, mean reward 0.1
Lifetime 17, episode 1022, mean reward 0.05
Lifetime 17, episode 